# OpenAI API - Chat Completion 완벽 가이드

이 노트북에서는 OpenAI의 Chat Completion API를 사용하여 대화형 AI를 구현하는 방법을 배웁니다.

**학습 목표:**
1. API 연결 및 기본 대화 요청
2. 대화의 문맥(Context)을 유지하는 방법
3. 실시간 응답을 위한 스트리밍(Streaming) 구현
4. 토큰(Token) 수 계산 및 비용 예측

In [2]:
from openai import OpenAI
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
api_key = os.getenv("openai_key")  # .env의 openai_key 값을 가져옴
client = OpenAI(api_key=api_key)

### 1. 기본 대화 및 문맥(Context) 이해

LLM은 기본적으로 '무상태(Stateless)'이므로, 이전 대화를 기억하지 못합니다. 따라서 이전 대화 내역(`messages`)을 계속 누적해서 전달해야 문맥을 유지할 수 있습니다.

In [5]:
response = client.chat.completions.create(
    model = 'gpt-4.1-mini',
    messages = [
        {
            'role': 'system',                    # 시스템 역할: 챗봇의 성격/규칙 지정
            'content': '너는 친절한 챗봇입니다.'  # 친절한 톤으로 응답하도록 지시
        },
        {
            'role': 'user',                      # 사용자 발화 1
            'content': '안녕, 나는 LLM을 공부하는 학생이야. 반갑다~'
        },
        {
            'role': 'assistant',                 # (이전 대화가 있었다는 가정) 어시스턴트의 답변을 히스토리로 제공
            'content': '안녕! 만나서 반가워~ LLM 공부하고 있다니 멋지다! 어떤 분야에 관심이 있는지 말해줄래? 내가 도움이 될 수 있으면 좋겠어.'
        },
        {
            'role': 'user',                      # 사용자 발화 2(질문)
            'content': 'gpt모델은 트랜스포머로부터 어떻게 발전된거야?'
        },
        {
            'role': 'assistant',                 # (이전 답변) 긴 설명을 히스토리로 제공
            'content': '''
좋은 질문이야! GPT 모델은 트랜스포머(Transformer) 아키텍처를 기반으로 발전했어. 간단히 설명할게.

1. **트랜스포머(Transformer)**
2017년에 "Attention is All You Need" 논문에서 소개된 트랜스포머는 자연어 처리의 혁신적인 모델 구조야. 이전 RNN이나 LSTM과 달리, 순차적인 계산 없이도 병렬 처리가 가능하고, '셀프 어텐션(Self-Attention)' 메커니즘을 사용해 문맥을 효율적으로 이해할 수 있어.

2. **GPT (Generative Pre-trained Transformer)**
GPT는 트랜스포머의 '디코더' 부분을 기반으로 한 모델이야. OpenAI가 처음 발표한 GPT는 대규모 텍스트 데이터에 대해 '언어 모델링' 방식으로 사전학습(pre-training)을 하고, 이후 특정 작업에 맞게 미세조정(fine-tuning)하는 방식을 사용했어.

3. **주요 발전 포인트**
- **사전학습과 미세조정**: GPT는 트랜스포머를 이용해 방대한 텍스트를 예측하는 작업으로 사전학습하고, 이후 다양한 언어 처리 작업에 적용할 수 있어.
- **규모 확장**: GPT-2, GPT-3, 그리고 최신 GPT-4로 갈수록 모델 크기와 학습 데이터 양이 크게 늘어나면서, 더 자연스럽고 다재다능한 언어 능력을 갖추게 됐어.
- **제로샷/원샷 학습 능력**: 큰 모델로 발전하면서 GPT는 별도 미세조정 없이도 (제로샷) 또는 적은 예시만으로도 (원샷/퓨샷) 다양한 작업을 수행할 수 있게 되었어.

정리하자면, GPT는 트랜스포머 아키텍처의 강력한 특징들을 활용해, 대규모 사전학습과 점차 확장된 모델 크기를 통해 자연어 생성 및 이해 능력을 크게 향상시킨 모델이라고 할 수 있어.

더 궁금한 점 있으면 언제든 물어봐!
'''
        },
        {
            'role': 'user',
            'content': '와 대단하구나! 세 줄로 요약해 줘.'
        }
    ],
    temperature = 1,  # 생성 다양성 (높을수록 창의적)
    top_p = 1,        # Top-p 샘플링 범위 (1이면 제한없음)
    max_completion_tokens = 4096  # 최대 출력 토큰 수
)

response

ChatCompletion(id='chatcmpl-D3bbCCocjqwZaBW98hkQeJEjrpYIi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='물론이지! 세 줄로 요약해 줄게.\n\n1. GPT는 트랜스포머 아키텍처의 디코더를 기반으로 한 대규모 언어 모델이야.  \n2. 방대한 텍스트로 사전학습하고, 다양한 언어 작업에 미세조정 또는 직접 활용 가능해.  \n3. 모델이 커질수록 더 자연스러운 언어 생성과 적은 예시로 학습하는 능력이 향상됐어.  \n\n더 궁금하면 언제든 알려줘!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769751886, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_e01c6f58e1', usage=CompletionUsage(completion_tokens=115, prompt_tokens=609, total_tokens=724, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
response.choices[0].message.content

'물론이지! 세 줄로 요약해 줄게.\n\n1. GPT는 트랜스포머 아키텍처의 디코더를 기반으로 한 대규모 언어 모델이야.  \n2. 방대한 텍스트로 사전학습하고, 다양한 언어 작업에 미세조정 또는 직접 활용 가능해.  \n3. 모델이 커질수록 더 자연스러운 언어 생성과 적은 예시로 학습하는 능력이 향상됐어.  \n\n더 궁금하면 언제든 알려줘!'

### 2. 실습: 대화형 챗봇 만들기

`while` 반복문을 사용하여 사용자의 입력을 계속 받고, 대화 히스토리를 업데이트하며 자연스러운 대화를 이어가는 간단한 챗봇을 구현해봅니다.

In [15]:
messages = [
    {'role': 'system', 'content': '당신은 지혜로운 챗봇입니다.'}
]

print('종료하려면, exit를 입력하세요\n')

while True:
    user_input = input('User: ')

    if user_input.lower().strip() == 'exit':
        print('대화를 종료합니다.')
        break
    
    print(f'User: {user_input}')
    messages.append({'role': 'user', 'content': user_input})

    response = client.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages = messages,
        temperature = 1,
        top_p = 1,
        max_completion_tokens = 4096
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')
    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요

Assistant: 오늘 날짜가 어떻게 되나요? 알려주시면 오늘이 무슨 날인지 알려드릴 수 있어요.
Assistant: 1월 30일에는 여러 의미 있는 일이 있어요. 예를 들면:

- 대한민국에서는 1월 30일이 특별한 공휴일은 아니지만,
- 역사적으로는 1948년 1월 30일에 인도 마하트마 간디가 암살당한 날로 기억돼요.

더 궁금한 점 있으면 알려주세요!
Assistant: 알려주셔서 감사합니다! 1월 30일이 금요일이라는 말씀이시군요. 올해 1월 30일이 금요일인지 확인하려면 연도를 알려주시면 정확히 알려드릴 수 있어요. 어떤 해의 1월 30일인지 알려주실 수 있을까요?
Assistant: 2026년 1월 30일은 금요일 맞습니다! 궁금한 점이나 필요하신 정보가 있으면 언제든지 말씀해 주세요.
Assistant: 반갑습니다, 홍길동님! 무엇을 도와드릴까요?
Assistant: 홍길동님이라고 하셨어요! 어떻게 도와드릴까요?
대화를 종료합니다.


- 서비스에서 대화 저장을 처리하는 방식
    - DB에 저장 (일반적)
        - RDB(PostgreSQL/MySQL) : 유저/세션/메시지 관계를 함께 저장
        - NoSQL(MongoDB 등) : 문서 형태로 대화를 한번에 저장
    - 캐시에 저장 (속도) + DB에 영구 저장(혼합)
        - Redis에 최근 N개의 메시지를 캐시로 두고 빠르게 응답
        - 일정 주기/이벤트 기준으로 DB에 영구 저장
    - 파일에 저장
        - JSON 파일 등으로 로컬에 저장
        - 붙여서 사용하는 가벼운 DB가 파일형태로 저장되는 경우

- 서비스 운영시에 고려사항
    - 전체 대화를 매번 모델에 다 보내게 되면 비용/속도 문제가 생긴다.
        - 최근 N개만 보낸다거나
        - 중간중간에 요약(summary)을 만들어 저장하고 요약본 + 최근 대화 형태로 컨텍스트 구성
    - 개인정보
        - DB 접근 암호화, 로그 마스킹
        - API/KEY 등은 대화메시지에 저장하지 않음
    - RAG 같은 검색형 기능들을 추가할 경우
        - 전체 대화를 DB에 저장 + 중요한 메시지는 벡터 DB에 임베딩 저장해서 예전에 대화했던 내용과 관련된 내용이 나오면 검색해서
          컨텍스트에 추가해준다.

### 3. 스트리밍(Streaming) 응답 처리

긴 답변을 한 번에 생성하려면 시간이 오래 걸립니다. `stream=True` 옵션을 사용하면 생성되는 토큰을 실시간으로 받아와서 타자기처럼 출력할 수 있습니다.

In [16]:
# 스트림처리 : 응답을 한 번에 받지 않고 조각(chunk) 단위로 실시간 수신
stream = client.chat.completions.create(
    model = 'gpt-4.1-mini',
    messages = [{'role': 'user', 'content': '나 스트림테스트 할 거니까, 아주 긴 답변을 해줘.'}],
    max_completion_tokens= 2000,
    stream=True    # 스트리밍 모드 활성화(True면 chunk 단위로 반환)
)

for chunk in stream:     # 스트림에서 chunk를 하나씩 순회하면서 수신
    content = chunk.choices[0].delta.content  # 이번 chunk에 새로 추가된 텍스트(delta)만 추출
    if content is not None:
        print(content, end='')  # 줄바꿈 없이 이어붙임

알겠어요! 아주 긴 답변을 원하신다니, 최대한 자세하고 길게 작성해볼게요. 이 답변을 통해 스트림 테스트에 충분한 데이터가 제공되길 바랍니다.

---

### 스트림 테스트용 아주 긴 답변

안녕하세요! 스트림 테스트를 하신다고 들었는데, 테스트 목적에 맞게 아주 긴 답변을 작성해 드리겠습니다. 아래 내용은 다양한 주제와 정보, 설명이 섞여 있어 충분히 긴 데이터를 포함하고 있습니다.

---

#### 1. 인공지능의 역사와 발전

인공지능(Artificial Intelligence, AI)은 컴퓨터와 기계가 인간처럼 사고하고 학습하는 능력을 갖추도록 하는 기술과 학문 분야입니다. AI의 역사는 1950년대 초반으로 거슬러 올라갑니다. 앨런 튜링(Alan Turing)은 1950년 “컴퓨팅 기계와 지능”이라는 논문에서 컴퓨터가 인간처럼 생각할 수 있는지를 시험하는 '튜링 테스트'를 제안하였고, 이는 AI 연구의 중요한 출발점이 되었습니다.

초기 AI 연구자들은 논리 기반 시스템과 규칙 기반 접근법을 주로 사용했습니다. 예를 들어, 엘리자(ELIZA)라는 프로그램은 간단한 대화형 시스템으로, 사람과 유사한 대화를 가능하게 했습니다. 하지만 초기 AI 시스템은 매우 제한적이었고 복잡한 문제 해결에는 어려움이 많았습니다.

1970~80년대에 접어들면서 '지식 기반 시스템'이나 '전문가 시스템'이 도입되어 특정 분야에서 높은 성과를 냈지만, 전반적인 AI 발전은 느린 편이었습니다. 컴퓨터의 연산 능력과 데이터 부족이 큰 제약 요인이었죠.

1990년대 이후로 컴퓨팅 파워가 급속도로 발전하고, 인터넷의 등장으로 방대한 데이터가 축적됨에 따라 AI 연구에 새로운 전기가 마련되었습니다. 특히 머신러닝과 딥러닝 기술이 빠르게 발전하면서 이미지 인식, 음성 인식, 자연어 처리 등 다양한 분야에서 성과를 내기 시작했습니다.

최근 몇 년간, OpenAI, 구글 딥마인드, 마이크로소프트, 페이스북 등 대형 IT 기업들은 거대한 언어 모델과 강화 학습, 생성형 AI 등을 통

### 스트리밍 응답으로 실시간 출력되는 콘솔 챗봇

In [17]:
messages = [
    {'role': 'system', 'content': '당신은 지혜로운 챗봇입니다.'}
]

print('종료하려면, exit를 입력하세요\n')

while True:
    user_input = input('User: ')

    if user_input.lower().strip() == 'exit':
        print('대화를 종료합니다.')
        break
    
    print(f'User: {user_input}')
    messages.append({'role': 'user', 'content': user_input})

    response = client.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages = messages,
        temperature = 1,
        top_p = 1,
        max_completion_tokens = 4096,
        stream=True
    )
    
    # 스트림 출력처리 : chunk가 도착할 때마다 바로 출력하고 전체 답변을 누적
    print(f'Assistant: ', end='')
    assistant_message = ''         # 전체 답변을 누적할 변수
    for chunk in response:         # 스트리밍 모드로 도착한 chunk 단위로 순회
        content = chunk.choices[0].delta.content  # 이번 chunk에 새로 추가된 텍스트(delta)만 추출
        if content is not None:
            print(content, end='')     # 줄바꿈 없이 실시간으로 이어서 출력
            assistant_message += content
    print()

    messages.append({'role': 'assistant', 'content': assistant_message})

종료하려면, exit를 입력하세요

User: 오늘 재밌는 일없나?
Assistant: 오늘 특별한 계획이 없더라도 소소하게 재미를 찾을 수 있어요! 예를 들어, 좋아하는 음악을 들으며 산책을 하거나, 새로운 레시피에 도전해보는 것도 좋고요. 아니면 짧은 영화나 드라마 한 편 보는 것도 재밌죠. 어떤 종류의 재미를 찾고 계신가요? 제가 추천해드릴게요!
User: 도파민
Assistant: 도파민에 대해 궁금하신가요? 도파민은 뇌에서 분비되는 신경전달물질로, 보상과 쾌락, 동기 부여에 중요한 역할을 해요. 우리가 즐거운 활동을 할 때 도파민이 분비되어 기분이 좋아지고, 더 그런 활동을 하도록 동기부여를 하죠.

오늘 도파민 분비를 촉진하는 재미있는 활동으로는 어떤 것들이 있을까요?

1. 운동하기: 가벼운 산책이나 스트레칭도 좋아요.
2. 음악 듣기: 좋아하는 노래를 들으면 기분이 좋아져요.
3. 새로운 것 배우기: 짧은 온라인 강의나 취미를 시작해보는 것도 도파민 분비에 도움이 돼요.
4. 작은 목표 달성하기: 할 일을 작게 나누어 하나씩 완료해 보세요.

원하시면 좀 더 구체적인 활동이나 정보를 알려드릴게요!
대화를 종료합니다.


### 4. 토큰(Token) 관리 및 비용 계산

LLM API 비용은 '토큰' 단위로 책정됩니다. `tiktoken` 라이브러리를 사용하여 정확한 토큰 수를 계산하고 예상 비용을 산출하는 방법을 알아봅니다.

In [18]:
%pip install tiktoken

   ---------------------------------------- 0.0/878.7 kB ? eta -:--:--
   ---------------------------------------- 878.7/878.7 kB 13.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import tiktoken  # 텍스트를 토큰 단위로 인코딩/계산하는 OpenAI 토크나이저 라이브러리

gpt35 = tiktoken.encoding_for_model('gpt-3.5')  # GPT3.5에 대응되는 인코딩(토크나이저) 가져옴
print(gpt35)
print(len(gpt35.encode('아버지가 방에 들어가십니다.')))  # 문장을 토큰으로 쪼갠 뒤 토큰 개수 확인

gpt4o = tiktoken.encoding_for_model('gpt-4o')
print(gpt4o)
print(len(gpt4o.encode('아버지가 방에 들어가십니다.')))

gpt41 = tiktoken.encoding_for_model('gpt-4.1')
print(gpt41)
print(len(gpt41.encode('아버지가 방에 들어가십니다.')))

# gpt52 = tiktoken.encoding_for_model('gpt-5.2')  # tiktoken에 매핑 정보가 없으면 KeyError가 발생

<Encoding 'cl100k_base'>
14
<Encoding 'o200k_base'>
10
<Encoding 'o200k_base'>
10


In [21]:
text = """
지난 22일 코스피 지수가 마침내 ‘꿈의 지수’인 5000포인트를 장중 돌파하며 새 역사를 썼습니다. 매일 신고가를 쓰는 코스피, 삼성전자, 현대차에 가려져있지만 국내 증시 못지 않게 오르는 자산이 ‘금’입니다.

국제 금값은 이달에만 14% 넘게 오르면서 22일(현지시간) 트라이온스당 4900달러를 돌파했습니다. 금 역시 역사적인 5000달러를 코앞에 두고 있죠. 국내 금값 역시 최근 역대 최고가를 경신하며 한 돈(3.75g)당 88만원 수준까지 올랐습니다. 은은 올해 37% 넘게 오르면서 99달러까지 오른 상태입니다.

국제 금값이 해외 투자은행(IB)이 지난해 예상한 수준을 단박에 넘어서면서 골드만삭스 등은 금값의 목표수준을 높이고 있습니다.

금값이 상승하는 배경엔 예상하기 어려운 도널드 트럼프 미국 대통령의 행보, 화폐가치 하락을 피해 자산을 보유하려는 투자자와 중앙은행의 매수세가 깔려있습니다. 최근 흐름을 짚어봤습니다.

올해 코스피가 초강세를 보이며 매 거래일마다 백단위 숫자가 바뀌었습니다. 최근 금값도 마찬가지입니다. 지난 16일 4500달러선에 머물던 금은 19일 4600달러, 20일 4700달러, 21일 4800달러, 22일 4900달러를 넘기며 4거래일 연속 앞자리를 바꾸는 강세를 이어갔습니다.

금은 올해 1월 20여일간 14.6% 가량 오른 겁니다. 그간 많이 올랐다는 필라델피아 반도체지수(13.7%)는 물론 세계 주요 주가지수를 압도하는 수익률입니다. 18% 오른 코스피를 제외하곤 금 투자가 수익률 측면에선 상당히 높았던 셈입니다. 지난 1년전과 비교해도 금값은 79.8%나 상승했습니다.

국내 금값도 23일엔 한국거래소에서 g당 2% 23만4100원에 마감해 역대 최고가를 경신했습니다. 금 한 돈당 87만7800원에 달합니다. 민간금거래소에선 이미 금 매입가가 한 돈당 100만원을 웃돕니다.

지난 10월엔 금투자 열풍이 불면서 ‘김치프리미엄(국내가격-국제가격)’이 두자릿 수에 달할 정도로 가격에 거품이 꼈었지만, 이번엔 다릅니다.

현재 김치프리미엄은 1% 남짓입니다. 당시 일일 거래대금이 5000억원을 웃돌고 개인이 매일 700억~1000억원 순매수에 나섰지만 최근엔 가격이 크게 뛰는데도 거래대금과 순매수액 모두 반토막인 상태입니다. 이번엔 국내 가격도 김치프리미엄이 아니라 정말로 비싸서 오른 것이죠.
"""

response = client.chat.completions.create(    # Chat Completion API 호출
    model = 'gpt-5.2',
    messages = [
        {"role": "system", "content": "너는 시사경제 전문가입니다. 주어진 뉴스기사의 핵심을 잘 요약/정리해주는 챗봇입니다. 사용자 입력값을 잘 읽고, 핵심내용 위주로 요약해주세요."},
        {"role": "user", 'content': text}
    ],
    response_format = {"type": "text"},  # 응답 형식
    temperature = 0.2,                   # 모델의 랜덤성/다양성 (낮으면 일관성, 높으면 창의적)
    max_completion_tokens = 4096,        # 최대 출력 토큰
    top_p = 1                            # 누적확률 p까지의 후보를 샘플링 (1이면 전체사용)
)

output = response.choices[0].message.content
output

'- **핵심 흐름**: 코스피가 5000포인트를 장중 돌파하며 주목받는 가운데, **금(과 은) 가격도 국내외에서 급등**하며 ‘또 다른 신고가 자산’으로 부상.\n\n- **국제 금값 급등**: 이달에만 **14% 이상 상승**, 22일(현지) **트라이온스당 4900달러 돌파**로 **5000달러**에 근접. 최근 4거래일 연속 4500→4600→4700→4800→4900달러로 ‘앞자리’가 바뀔 정도로 강세.\n\n- **국내 금값도 최고치 경신**: 23일 한국거래소에서 **g당 23만4100원(종가 기준)**으로 역대 최고가. **한 돈(3.75g) 약 87만7800원~88만원 수준**. 민간 거래소에선 **매입가가 한 돈 100만원 상회** 사례도 언급.\n\n- **은도 강세**: 은 가격은 **올해 37% 넘게 상승**, **99달러** 수준까지 상승.\n\n- **수익률 비교(상대적 강세 강조)**: 금은 1월 들어 약 **14.6% 상승**해 필라델피아반도체지수(13.7%) 등 주요 지수 대비 강했고, **코스피(18% 상승)를 제외하면** 두드러진 성과로 평가. **1년 전 대비 금값 79.8% 상승**.\n\n- **전망 상향**: 금값이 해외 IB들의 기존 예상치를 빠르게 상회하면서 **골드만삭스 등은 목표가를 상향**하는 분위기.\n\n- **상승 배경(기사의 해석)**:  \n  1) **트럼프 대통령의 예측 어려운 행보**에 따른 불확실성 확대  \n  2) **화폐가치 하락(인플레이션/통화가치 방어) 우려** 속 실물자산 선호  \n  3) **중앙은행 및 투자자 매수세** 유입\n\n- **‘이번엔 김치프리미엄 때문이 아니다’가 포인트**:  \n  - 작년 10월엔 금 투자 열풍으로 **김치프리미엄이 두 자릿수**까지 붙고 거래대금·개인 순매수가 과열(일 거래대금 5000억+, 개인 700억~1000억 순매수)됐지만,  \n  - 현재는 **김치프리미엄이 1% 내외**로 낮고, **거래대금·순매수도 반토막**.  \n  → 즉

In [23]:
model = 'gpt-4.1-mini'

try:
    enc = tiktoken.encoding_for_model(model)        # model에 매핑된 인코딩(토크나이저) 가져옴
except KeyError as e:
    print(f'Warning: {model}을 찾을 수 없습니다.')
    enc = tiktoken.get_encoding('o200k_base')

input_token_len = len(enc.encode(text))    # 입력(text)의 토큰 개수 계산
output_token_len = len(enc.encode(output))   # 출력(output)의 토큰 개수 계산
print(f'input_token_len = {input_token_len}')
print(f'output_token_len = {output_token_len}')

input_token_len = 742
output_token_len = 648


In [24]:
# gpt 5.2 기준 input/output price
input_price = 0.875
output_price = 7

input_cost = (input_price / 1_000_000) * input_token_len    # 1회 호출 기준 입력 비용
output_cost = (output_price / 1_000_000) * input_token_len  # 1회 호출 기준 출력 비용
total_cost = input_cost + output_cost                       # 1회 호출 기준 총 비용
print("input_cost =", input_cost)
print("output_cost =", output_cost)
print("total_cost =", total_cost)

input_cost = 0.00064925
output_cost = 0.005194
total_cost = 0.0058432499999999995


In [25]:
num_service_call = 10_000_000                       # 월 호출 횟수 가정
monthly_total_cost = total_cost * num_service_call
print("monthly_total_cost =", monthly_total_cost)   # 월 총 비용

monthly_total_cost = 58432.49999999999
